In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import matplotlib.backends.backend_pdf

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

%matplotlib inline

dir_data='/Users/ahenny/'
dir1='/Volumes/LaCie/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/day/'

In [ ]:
ds=xr.open_dataset('/Users/ahenny/Downloads/full_data_daily_v2022_10_'+str(1982)+'.nc')
field_interp=ds['precip'].transpose('time','lat','lon')[0,:,:]
lats_sel=[x for x in field_interp.lat.values if -60<=x<=70]
field_interp=field_interp.sel(lat=lats_sel)

In [ ]:
yrs=np.arange(1990,2023,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)

    if 1980<=year<=2020:
    
        ds=xr.open_dataset(dir2+'mswep_past_deres_'+str(year)+'.nc')
        precip=ds['precip']
        
        lons_east=[x for x in precip.lon.values if x>=90]
        lons_west=[x for x in precip.lon.values if x<-90]
        precip_east=precip.sel(lon=lons_east)
        precip_west=precip.sel(lon=lons_west)
        precip_east['lon']=[x-360 for x in lons_east]
        precip_west['lon']=[x+360 for x in lons_west]
        precip=xr.concat([precip_east,precip,precip_west],dim='lon')
        
        precip=precip.interp_like(field_interp)

        defined1=xr.ones_like(precip).sum(dim='time')
        defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
        if year==1990:
            max_concat_mswep=defined1
            present_concat_mswep=defined
        else:
            max_concat_mswep=xr.concat([max_concat_mswep,defined1],dim='time')
            present_concat_mswep=xr.concat([present_concat_mswep,defined],dim='time')
        ds.close()
        precip.close()
    
    if year>=1983:
        ds=xr.open_dataset(dir_data+'holding/persiann_cdr_'+str(year)+'.nc')
        precip=ds['precip']
    
        lons_east=[x for x in precip.lon.values if x>=180.]
        lons_west=[x for x in precip.lon.values if x<180.]
        p_east=precip.sel(lon=lons_east)
        p_west=precip.sel(lon=lons_west)
        p_east['lon']=[x-360. for x in lons_east]
        precip=xr.concat([p_east,p_west],dim='lon')
        p_east.close()
        p_west.close()
        
        lons_east=[x for x in precip.lon.values if x>=90]
        lons_west=[x for x in precip.lon.values if x<-90]
        precip_east=precip.sel(lon=lons_east)
        precip_west=precip.sel(lon=lons_west)
        precip_east['lon']=[x-360 for x in lons_east]
        precip_west['lon']=[x+360 for x in lons_west]
        precip=xr.concat([precip_east,precip,precip_west],dim='lon')
        
        precip=precip.interp_like(field_interp)
    
        defined1=xr.ones_like(precip).sum(dim='time')
        defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
        if year==1990:
            max_concat_persiann=defined1
            present_concat_persiann=defined
        else:
            max_concat_persiann=xr.concat([max_concat_persiann,defined1],dim='time')
            present_concat_persiann=xr.concat([present_concat_persiann,defined],dim='time')
            
        ds.close()
        precip.close()
    
    if year<=2016:
    
        ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_'+str(year)+'.nc')
        precip=ds['p']
        
        lons_east=[x for x in precip.lon.values if x>=90]
        lons_west=[x for x in precip.lon.values if x<-90]
        precip_east=precip.sel(lon=lons_east)
        precip_west=precip.sel(lon=lons_west)
        precip_east['lon']=[x-360 for x in lons_east]
        precip_west['lon']=[x+360 for x in lons_west]
        precip=xr.concat([precip_east,precip,precip_west],dim='lon')
        
        precip=precip.interp_like(field_interp)
        
        defined1=xr.ones_like(precip).sum(dim='time')
        defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
        if year==1990:
            max_concat_regen=defined1
            present_concat_regen=defined
        else:
            max_concat_regen=xr.concat([max_concat_regen,defined1],dim='time')
            present_concat_regen=xr.concat([present_concat_regen,defined],dim='time')
            
        ds.close()
        precip.close()
        
    if 1982<=year<=2020:
        
        ds=xr.open_dataset('/Users/ahenny/Downloads/full_data_daily_v2022_10_'+str(year)+'.nc')
        precip=ds['precip'].transpose('time','lat','lon')
        
        lons_east=[x for x in precip.lon.values if x>=90]
        lons_west=[x for x in precip.lon.values if x<-90]
        precip_east=precip.sel(lon=lons_east)
        precip_west=precip.sel(lon=lons_west)
        precip_east['lon']=[x-360 for x in lons_east]
        precip_west['lon']=[x+360 for x in lons_west]
        precip=xr.concat([precip_east,precip,precip_west],dim='lon')
        
        precip=precip.interp_like(field_interp)
        
        defined1=xr.ones_like(precip).sum(dim='time')
        defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
        if year==1990:
            max_concat_gpcc=defined1
            present_concat_gpcc=defined
        else:
            max_concat_gpcc=xr.concat([max_concat_gpcc,defined1],dim='time')
            present_concat_gpcc=xr.concat([present_concat_gpcc,defined],dim='time')
    
        ds.close()
        precip.close()
        
    ds=xr.open_dataset(dir2+'precip.'+str(year)+'.nc')
    precip=ds['precip']
    
    lons_east=[x for x in precip.lon.values if x>=180.]
    lons_west=[x for x in precip.lon.values if x<180.]
    p_east=precip.sel(lon=lons_east)
    p_west=precip.sel(lon=lons_west)
    p_east['lon']=[x-360. for x in lons_east]
    precip=xr.concat([p_east,p_west],dim='lon')
    p_east.close()
    p_west.close()
    
    lons_east=[x for x in precip.lon.values if x>=90]
    lons_west=[x for x in precip.lon.values if x<-90]
    precip_east=precip.sel(lon=lons_east)
    precip_west=precip.sel(lon=lons_west)
    precip_east['lon']=[x-360 for x in lons_east]
    precip_west['lon']=[x+360 for x in lons_west]
    precip=xr.concat([precip_east,precip,precip_west],dim='lon')
    
    precip=precip.interp_like(field_interp)
    
    defined1=xr.ones_like(precip).sum(dim='time')
    defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
    if year==1990:
        max_concat_cpc=defined1
        present_concat_cpc=defined
    else:
        max_concat_cpc=xr.concat([max_concat_cpc,defined1],dim='time')
        present_concat_cpc=xr.concat([present_concat_cpc,defined],dim='time')
    
    ds.close()
    precip.close()
        
    ds=xr.open_dataset(dir_data+'holding/era_5_precip_'+str(year)+'.nc')
    precip=ds['precip']
    
    lons_east=[x for x in precip.lon.values if x>=90]
    lons_west=[x for x in precip.lon.values if x<-90]
    precip_east=precip.sel(lon=lons_east)
    precip_west=precip.sel(lon=lons_west)
    precip_east['lon']=[x-360 for x in lons_east]
    precip_west['lon']=[x+360 for x in lons_west]
    precip=xr.concat([precip_east,precip,precip_west],dim='lon')
    
    precip=precip.interp_like(field_interp)
    
    defined1=xr.ones_like(precip).sum(dim='time')
    defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
    if year==1990:
        max_concat_era5=defined1
        present_concat_era5=defined
    else:
        max_concat_era5=xr.concat([max_concat_era5,defined1],dim='time')
        present_concat_era5=xr.concat([present_concat_era5,defined],dim='time')
    
    ds.close()
    precip.close()
    
    ds=xr.open_dataset(dir2+'jra55_precip_'+str(year)+'.nc')
    precip=ds['precip']
    
    lons_east=[x for x in precip.lon.values if x>=90]
    lons_west=[x for x in precip.lon.values if x<-90]
    precip_east=precip.sel(lon=lons_east)
    precip_west=precip.sel(lon=lons_west)
    precip_east['lon']=[x-360 for x in lons_east]
    precip_west['lon']=[x+360 for x in lons_west]
    precip=xr.concat([precip_east,precip,precip_west],dim='lon')
    
    precip=precip.interp_like(field_interp)
    
    defined1=xr.ones_like(precip).sum(dim='time')
    defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
    if year==1990:
        max_concat_jra55=defined1
        present_concat_jra55=defined
    else:
        max_concat_jra55=xr.concat([max_concat_jra55,defined1],dim='time')
        present_concat_jra55=xr.concat([present_concat_jra55,defined],dim='time')
    
    ds.close()
    precip.close()
    
    ds=xr.open_dataset(dir_data+'holding/merra2_precip_'+str(year)+'.nc')
    precip=ds['precip']
    
    lons_east=[x for x in precip.lon.values if x>=90]
    lons_west=[x for x in precip.lon.values if x<-90]
    precip_east=precip.sel(lon=lons_east)
    precip_west=precip.sel(lon=lons_west)
    precip_east['lon']=[x-360 for x in lons_east]
    precip_west['lon']=[x+360 for x in lons_west]
    precip=xr.concat([precip_east,precip,precip_west],dim='lon')
    
    precip=precip.interp_like(field_interp)
    
    defined1=xr.ones_like(precip).sum(dim='time')
    defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
    if year==1990:
        max_concat_merra2=defined1
        present_concat_merra2=defined
    else:
        max_concat_merra2=xr.concat([max_concat_merra2,defined1],dim='time')
        present_concat_merra2=xr.concat([present_concat_merra2,defined],dim='time')
    
    ds.close()
    precip.close()

In [ ]:
max_all=max_concat_era5.sum(dim='time')
present_all=present_concat_era5.sum(dim='time')
ratio_all_era5=present_all/max_all

max_all=max_concat_jra55.sum(dim='time')
present_all=present_concat_jra55.sum(dim='time')
ratio_all_jra55=present_all/max_all

max_all=max_concat_merra2.sum(dim='time')
present_all=present_concat_merra2.sum(dim='time')
ratio_all_merra2=present_all/max_all

max_all=max_concat_cpc.sum(dim='time')
present_all=present_concat_cpc.sum(dim='time')
ratio_all_cpc=present_all/max_all

max_all=max_concat_gpcc.sum(dim='time')
present_all=present_concat_gpcc.sum(dim='time')
ratio_all_gpcc=present_all/max_all

max_all=max_concat_regen.sum(dim='time')
present_all=present_concat_regen.sum(dim='time')
ratio_all_regen=present_all/max_all

max_all=max_concat_persiann.sum(dim='time')
present_all=present_concat_persiann.sum(dim='time')
ratio_all_persiann=present_all/max_all

max_all=max_concat_mswep.sum(dim='time')
present_all=present_concat_mswep.sum(dim='time')
ratio_all_mswep=present_all/max_all

In [ ]:
test=ratio_all_persiann
print(test)

fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(test.lon,test.lat,test,vmin=0,vmax=1,cmap=plt.cm.seismic)
ax.coastlines(resolution='10m')
plt.show()

In [ ]:
yrs=np.arange(1990,2023,1)

ds=xr.open_dataset(dir_data+'mswep_threshold_r95p.nc')
threshold_mswep_95=ds['threshold'].interp_like(field_interp)
ds=xr.open_dataset(dir_data+'mswep_threshold_r99p.nc')
threshold_mswep_99=ds['threshold'].interp_like(field_interp)
ds=xr.open_dataset(dir_data+'mswep_threshold_final_98_all_30.nc')
threshold_mswep_98=ds['threshold'].interp_like(field_interp)

ds=xr.open_dataset(dir_data+'persiann_threshold_r95p.nc')
threshold_persiann_95=ds['threshold'].interp_like(field_interp)
ds=xr.open_dataset(dir_data+'persiann_threshold_r99p.nc')
threshold_persiann_99=ds['threshold'].interp_like(field_interp)
ds=xr.open_dataset(dir_data+'persiann_threshold_final_98_all.nc')
threshold_persiann_98=ds['threshold'].interp_like(field_interp)

ds=xr.open_dataset(dir_data+'regenfull_threshold_r95p.nc')
threshold_regen_95=ds['threshold'].interp_like(field_interp)
ds=xr.open_dataset(dir_data+'regenfull_threshold_r99p.nc')
threshold_regen_99=ds['threshold'].interp_like(field_interp)
ds=xr.open_dataset(dir_data+'regenfull_threshold_final_98_all.nc')
threshold_regen_98=ds['threshold'].interp_like(field_interp)

ds=xr.open_dataset(dir_data+'cpc_threshold_r95p.nc')
threshold_cpc_95=ds['threshold'].interp_like(field_interp)
ds=xr.open_dataset(dir_data+'cpc_threshold_r99p.nc')
threshold_cpc_99=ds['threshold'].interp_like(field_interp)
ds=xr.open_dataset(dir_data+'cpc_threshold_final_98_all.nc')
threshold_cpc_98=ds['threshold'].interp_like(field_interp)

ds=xr.open_dataset(dir_data+'gpcc_threshold_r95p.nc')
threshold_gpcc_95=ds['threshold'].interp_like(field_interp)
ds=xr.open_dataset(dir_data+'gpcc_threshold_r99p.nc')
threshold_gpcc_99=ds['threshold'].interp_like(field_interp)
ds=xr.open_dataset(dir_data+'gpcc_threshold_final_98_all.nc')
threshold_gpcc_98=ds['threshold'].interp_like(field_interp)

ds=xr.open_dataset(dir_data+'era5_threshold_r95p.nc')
threshold_era5_95=ds['threshold'].interp_like(field_interp)
ds=xr.open_dataset(dir_data+'era5_threshold_r99p.nc')
threshold_era5_99=ds['threshold'].interp_like(field_interp)
ds=xr.open_dataset(dir_data+'era5_threshold_final_98_all.nc')
threshold_era5_98=ds['threshold'].interp_like(field_interp)

ds=xr.open_dataset(dir_data+'jra55_threshold_r95p.nc')
threshold_jra55_95=ds['threshold'].interp_like(field_interp)
ds=xr.open_dataset(dir_data+'jra55_threshold_r99p.nc')
threshold_jra55_99=ds['threshold'].interp_like(field_interp)
ds=xr.open_dataset(dir_data+'jra55_threshold_final_98_all.nc')
threshold_jra55_98=ds['threshold'].interp_like(field_interp)

ds=xr.open_dataset(dir_data+'merra2_threshold_r95p.nc')
threshold_merra2_95=ds['threshold'].interp_like(field_interp)
ds=xr.open_dataset(dir_data+'merra2_threshold_r99p.nc')
threshold_merra2_99=ds['threshold'].interp_like(field_interp)
ds=xr.open_dataset(dir_data+'merra2_threshold_final_98_all.nc')
threshold_merra2_98=ds['threshold'].interp_like(field_interp)

for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    
    if 1980<=year<=2020:
    
        ds=xr.open_dataset(dir2+'mswep_past_deres_'+str(year)+'.nc')
        precip=ds['precip']
        
        lons_east=[x for x in precip.lon.values if x>=90]
        lons_west=[x for x in precip.lon.values if x<-90]
        precip_east=precip.sel(lon=lons_east)
        precip_west=precip.sel(lon=lons_west)
        precip_east['lon']=[x-360 for x in lons_east]
        precip_west['lon']=[x+360 for x in lons_west]
        precip=xr.concat([precip_east,precip,precip_west],dim='lon')
        
        precip=precip.interp_like(field_interp)

        threshold_new=xr.zeros_like(precip)
        for k in range(threshold_new.time.size):
            date=pd.to_datetime(threshold_new.time.values[k])
            if date.month==2 and date.day==29:
                threshold_new1=threshold_mswep_98.sel(time=dt.datetime(1981,2,28))
                threshold_new2=threshold_mswep_98.sel(time=dt.datetime(1981,3,1))
                threshold_new3=(threshold_new1+threshold_new1)/2.
            else:
                threshold_new3=threshold_mswep_98.sel(time=dt.datetime(1981,date.month,date.day))

            threshold_new[k,:,:]=threshold_new3.values

        defined1=xr.ones_like(precip).sum(dim='time')
        defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
        ratio=defined/defined1
        ratio=ratio.where(ratio>0)

        precip_sel=precip.where(precip>=1.)
        ones_sel=xr.ones_like(precip).where(precip>=1.)
        precip_sel_95=precip_sel.where(precip>=threshold_mswep_95)
        ones_sel_95=ones_sel.where(precip>=threshold_mswep_95)
        precip_sel_99=precip_sel.where(precip>=threshold_mswep_99)
        ones_sel_99=ones_sel.where(precip>=threshold_mswep_99)
        precip_sel_98=precip_sel.where(precip>=threshold_new)
        ones_sel_98=ones_sel.where(precip>=threshold_new)

        precip_total_95=precip_sel_95.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_mswep>=0.9)/ratio
        freq_total_95=ones_sel_95.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_mswep>=0.9)/ratio
        intensity_sel_95=precip_sel_95.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_mswep>=0.9)
        precip_total_98=precip_sel_98.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_mswep>=0.9)/ratio
        freq_total_98=ones_sel_98.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_mswep>=0.9)/ratio
        intensity_sel_98=precip_sel_98.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_mswep>=0.9)
        precip_total_99=precip_sel_99.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_mswep>=0.9)/ratio
        freq_total_99=ones_sel_99.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_mswep>=0.9)/ratio
        intensity_sel_99=precip_sel_99.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_mswep>=0.9)

        if year==1990:
            total_concat_mswep_95=precip_total_95
            freq_concat_mswep_95=freq_total_95
            intensity_concat_mswep_95=intensity_sel_95
            total_concat_mswep_98=precip_total_98
            freq_concat_mswep_98=freq_total_98
            intensity_concat_mswep_98=intensity_sel_98
            total_concat_mswep_99=precip_total_99
            freq_concat_mswep_99=freq_total_99
            intensity_concat_mswep_99=intensity_sel_99

        else:
            total_concat_mswep_95=xr.concat([total_concat_mswep_95,precip_total_95],dim='time')
            freq_concat_mswep_95=xr.concat([freq_concat_mswep_95,freq_total_95],dim='time')
            intensity_concat_mswep_95=xr.concat([intensity_concat_mswep_95,intensity_sel_95],dim='time')
            total_concat_mswep_98=xr.concat([total_concat_mswep_98,precip_total_98],dim='time')
            freq_concat_mswep_98=xr.concat([freq_concat_mswep_98,freq_total_98],dim='time')
            intensity_concat_mswep_98=xr.concat([intensity_concat_mswep_98,intensity_sel_98],dim='time')
            total_concat_mswep_99=xr.concat([total_concat_mswep_99,precip_total_99],dim='time')
            freq_concat_mswep_99=xr.concat([freq_concat_mswep_99,freq_total_99],dim='time')
            intensity_concat_mswep_99=xr.concat([intensity_concat_mswep_99,intensity_sel_99],dim='time')

        
        
        
        ds.close()
        precip.close()
    
    if year>=1983:
        ds=xr.open_dataset(dir_data+'holding/persiann_cdr_'+str(year)+'.nc')
        precip=ds['precip']
    
        lons_east=[x for x in precip.lon.values if x>=180.]
        lons_west=[x for x in precip.lon.values if x<180.]
        p_east=precip.sel(lon=lons_east)
        p_west=precip.sel(lon=lons_west)
        p_east['lon']=[x-360. for x in lons_east]
        precip=xr.concat([p_east,p_west],dim='lon')
        p_east.close()
        p_west.close()
        
        lons_east=[x for x in precip.lon.values if x>=90]
        lons_west=[x for x in precip.lon.values if x<-90]
        precip_east=precip.sel(lon=lons_east)
        precip_west=precip.sel(lon=lons_west)
        precip_east['lon']=[x-360 for x in lons_east]
        precip_west['lon']=[x+360 for x in lons_west]
        precip=xr.concat([precip_east,precip,precip_west],dim='lon')
        
        precip=precip.interp_like(field_interp)
    
        threshold_new=xr.zeros_like(precip)
        for k in range(threshold_new.time.size):
            date=pd.to_datetime(threshold_new.time.values[k])
            if date.month==2 and date.day==29:
                threshold_new1=threshold_persiann_98.sel(time=dt.datetime(1983,2,28))
                threshold_new2=threshold_persiann_98.sel(time=dt.datetime(1983,3,1))
                threshold_new3=(threshold_new1+threshold_new1)/2.
            else:
                threshold_new3=threshold_persiann_98.sel(time=dt.datetime(1983,date.month,date.day))

            threshold_new[k,:,:]=threshold_new3.values

        defined1=xr.ones_like(precip).sum(dim='time')
        defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
        ratio=defined/defined1
        ratio=ratio.where(ratio>0)

        precip_sel=precip.where(precip>=1.)
        ones_sel=xr.ones_like(precip).where(precip>=1.)
        precip_sel_95=precip_sel.where(precip>=threshold_mswep_95)
        ones_sel_95=ones_sel.where(precip>=threshold_mswep_95)
        precip_sel_99=precip_sel.where(precip>=threshold_mswep_99)
        ones_sel_99=ones_sel.where(precip>=threshold_mswep_99)
        precip_sel_98=precip_sel.where(precip>=threshold_new)
        ones_sel_98=ones_sel.where(precip>=threshold_new)

        precip_total_95=precip_sel_95.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_persiann>=0.9)/ratio
        freq_total_95=ones_sel_95.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_persiann>=0.9)/ratio
        intensity_sel_95=precip_sel_95.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_persiann>=0.9)
        precip_total_98=precip_sel_98.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_persiann>=0.9)/ratio
        freq_total_98=ones_sel_98.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_persiann>=0.9)/ratio
        intensity_sel_98=precip_sel_98.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_persiann>=0.9)
        precip_total_99=precip_sel_99.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_persiann>=0.9)/ratio
        freq_total_99=ones_sel_99.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_persiann>=0.9)/ratio
        intensity_sel_99=precip_sel_99.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_persiann>=0.9)

        if year==1990:
            total_concat_persiann_95=precip_total_95
            freq_concat_persiann_95=freq_total_95
            intensity_concat_persiann_95=intensity_sel_95
            total_concat_persiann_98=precip_total_98
            freq_concat_persiann_98=freq_total_98
            intensity_concat_persiann_98=intensity_sel_98
            total_concat_persiann_99=precip_total_99
            freq_concat_persiann_99=freq_total_99
            intensity_concat_persiann_99=intensity_sel_99

        else:
            total_concat_persiann_95=xr.concat([total_concat_persiann_95,precip_total_95],dim='time')
            freq_concat_persiann_95=xr.concat([freq_concat_persiann_95,freq_total_95],dim='time')
            intensity_concat_persiann_95=xr.concat([intensity_concat_persiann_95,intensity_sel_95],dim='time')
            total_concat_persiann_98=xr.concat([total_concat_persiann_98,precip_total_98],dim='time')
            freq_concat_persiann_98=xr.concat([freq_concat_persiann_98,freq_total_98],dim='time')
            intensity_concat_persiann_98=xr.concat([intensity_concat_persiann_98,intensity_sel_98],dim='time')
            total_concat_persiann_99=xr.concat([total_concat_persiann_99,precip_total_99],dim='time')
            freq_concat_persiann_99=xr.concat([freq_concat_persiann_99,freq_total_99],dim='time')
            intensity_concat_persiann_99=xr.concat([intensity_concat_persiann_99,intensity_sel_99],dim='time')

        
        ds.close()
        precip.close()
    
    if year<=2016:
    
        ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_'+str(year)+'.nc')
        precip=ds['p']
        
        lons_east=[x for x in precip.lon.values if x>=90]
        lons_west=[x for x in precip.lon.values if x<-90]
        precip_east=precip.sel(lon=lons_east)
        precip_west=precip.sel(lon=lons_west)
        precip_east['lon']=[x-360 for x in lons_east]
        precip_west['lon']=[x+360 for x in lons_west]
        precip=xr.concat([precip_east,precip,precip_west],dim='lon')
        
        precip=precip.interp_like(field_interp)
        
        threshold_new=xr.zeros_like(precip)
        for k in range(threshold_new.time.size):
            date=pd.to_datetime(threshold_new.time.values[k])
            if date.month==2 and date.day==29:
                threshold_new1=threshold_regen_98.sel(time=dt.datetime(1981,2,28))
                threshold_new2=threshold_regen_98.sel(time=dt.datetime(1981,3,1))
                threshold_new3=(threshold_new1+threshold_new1)/2.
            else:
                threshold_new3=threshold_regen_98.sel(time=dt.datetime(1981,date.month,date.day))

            threshold_new[k,:,:]=threshold_new3.values

        defined1=xr.ones_like(precip).sum(dim='time')
        defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
        ratio=defined/defined1
        ratio=ratio.where(ratio>0)

        precip_sel=precip.where(precip>=1.)
        ones_sel=xr.ones_like(precip).where(precip>=1.)
        precip_sel_95=precip_sel.where(precip>=threshold_mswep_95)
        ones_sel_95=ones_sel.where(precip>=threshold_mswep_95)
        precip_sel_99=precip_sel.where(precip>=threshold_mswep_99)
        ones_sel_99=ones_sel.where(precip>=threshold_mswep_99)
        precip_sel_98=precip_sel.where(precip>=threshold_new)
        ones_sel_98=ones_sel.where(precip>=threshold_new)

        precip_total_95=precip_sel_95.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_regen>=0.9)/ratio
        freq_total_95=ones_sel_95.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_regen>=0.9)/ratio
        intensity_sel_95=precip_sel_95.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_regen>=0.9)
        precip_total_98=precip_sel_98.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_regen>=0.9)/ratio
        freq_total_98=ones_sel_98.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_regen>=0.9)/ratio
        intensity_sel_98=precip_sel_98.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_regen>=0.9)
        precip_total_99=precip_sel_99.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_regen>=0.9)/ratio
        freq_total_99=ones_sel_99.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_regen>=0.9)/ratio
        intensity_sel_99=precip_sel_99.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_regen>=0.9)

        if year==1990:
            total_concat_regen_95=precip_total_95
            freq_concat_regen_95=freq_total_95
            intensity_concat_regen_95=intensity_sel_95
            total_concat_regen_98=precip_total_98
            freq_concat_regen_98=freq_total_98
            intensity_concat_regen_98=intensity_sel_98
            total_concat_regen_99=precip_total_99
            freq_concat_regen_99=freq_total_99
            intensity_concat_regen_99=intensity_sel_99

        else:
            total_concat_regen_95=xr.concat([total_concat_regen_95,precip_total_95],dim='time')
            freq_concat_regen_95=xr.concat([freq_concat_regen_95,freq_total_95],dim='time')
            intensity_concat_regen_95=xr.concat([intensity_concat_regen_95,intensity_sel_95],dim='time')
            total_concat_regen_98=xr.concat([total_concat_regen_98,precip_total_98],dim='time')
            freq_concat_regen_98=xr.concat([freq_concat_regen_98,freq_total_98],dim='time')
            intensity_concat_regen_98=xr.concat([intensity_concat_regen_98,intensity_sel_98],dim='time')
            total_concat_regen_99=xr.concat([total_concat_regen_99,precip_total_99],dim='time')
            freq_concat_regen_99=xr.concat([freq_concat_regen_99,freq_total_99],dim='time')
            intensity_concat_regen_99=xr.concat([intensity_concat_regen_99,intensity_sel_99],dim='time')

        
        ds.close()
        precip.close()
        
    if 1982<=year<=2020:
        
        ds=xr.open_dataset('/Users/ahenny/Downloads/full_data_daily_v2022_10_'+str(year)+'.nc')
        precip=ds['precip'].transpose('time','lat','lon')
        
        lons_east=[x for x in precip.lon.values if x>=90]
        lons_west=[x for x in precip.lon.values if x<-90]
        precip_east=precip.sel(lon=lons_east)
        precip_west=precip.sel(lon=lons_west)
        precip_east['lon']=[x-360 for x in lons_east]
        precip_west['lon']=[x+360 for x in lons_west]
        precip=xr.concat([precip_east,precip,precip_west],dim='lon')
        
        precip=precip.interp_like(field_interp)
        
        threshold_new=xr.zeros_like(precip)
        for k in range(threshold_new.time.size):
            date=pd.to_datetime(threshold_new.time.values[k])
            if date.month==2 and date.day==29:
                threshold_new1=threshold_gpcc_98.sel(time=dt.datetime(1983,2,28))
                threshold_new2=threshold_gpcc_98.sel(time=dt.datetime(1983,3,1))
                threshold_new3=(threshold_new1+threshold_new1)/2.
            else:
                threshold_new3=threshold_gpcc_98.sel(time=dt.datetime(1983,date.month,date.day))

            threshold_new[k,:,:]=threshold_new3.values

        defined1=xr.ones_like(precip).sum(dim='time')
        defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
        ratio=defined/defined1
        ratio=ratio.where(ratio>0)

        precip_sel=precip.where(precip>=1.)
        ones_sel=xr.ones_like(precip).where(precip>=1.)
        precip_sel_95=precip_sel.where(precip>=threshold_mswep_95)
        ones_sel_95=ones_sel.where(precip>=threshold_mswep_95)
        precip_sel_99=precip_sel.where(precip>=threshold_mswep_99)
        ones_sel_99=ones_sel.where(precip>=threshold_mswep_99)
        precip_sel_98=precip_sel.where(precip>=threshold_new)
        ones_sel_98=ones_sel.where(precip>=threshold_new)

        precip_total_95=precip_sel_95.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_gpcc>=0.9)/ratio
        freq_total_95=ones_sel_95.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_gpcc>=0.9)/ratio
        intensity_sel_95=precip_sel_95.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_gpcc>=0.9)
        precip_total_98=precip_sel_98.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_gpcc>=0.9)/ratio
        freq_total_98=ones_sel_98.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_gpcc>=0.9)/ratio
        intensity_sel_98=precip_sel_98.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_gpcc>=0.9)
        precip_total_99=precip_sel_99.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_gpcc>=0.9)/ratio
        freq_total_99=ones_sel_99.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_gpcc>=0.9)/ratio
        intensity_sel_99=precip_sel_99.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_gpcc>=0.9)

        if year==1990:
            total_concat_gpcc_95=precip_total_95
            freq_concat_gpcc_95=freq_total_95
            intensity_concat_gpcc_95=intensity_sel_95
            total_concat_gpcc_98=precip_total_98
            freq_concat_gpcc_98=freq_total_98
            intensity_concat_gpcc_98=intensity_sel_98
            total_concat_gpcc_99=precip_total_99
            freq_concat_gpcc_99=freq_total_99
            intensity_concat_gpcc_99=intensity_sel_99

        else:
            total_concat_gpcc_95=xr.concat([total_concat_gpcc_95,precip_total_95],dim='time')
            freq_concat_gpcc_95=xr.concat([freq_concat_gpcc_95,freq_total_95],dim='time')
            intensity_concat_gpcc_95=xr.concat([intensity_concat_gpcc_95,intensity_sel_95],dim='time')
            total_concat_gpcc_98=xr.concat([total_concat_gpcc_98,precip_total_98],dim='time')
            freq_concat_gpcc_98=xr.concat([freq_concat_gpcc_98,freq_total_98],dim='time')
            intensity_concat_gpcc_98=xr.concat([intensity_concat_gpcc_98,intensity_sel_98],dim='time')
            total_concat_gpcc_99=xr.concat([total_concat_gpcc_99,precip_total_99],dim='time')
            freq_concat_gpcc_99=xr.concat([freq_concat_gpcc_99,freq_total_99],dim='time')
            intensity_concat_gpcc_99=xr.concat([intensity_concat_gpcc_99,intensity_sel_99],dim='time')

        
        ds.close()
        precip.close()
        
    ds=xr.open_dataset(dir2+'precip.'+str(year)+'.nc')
    precip=ds['precip']
    
    lons_east=[x for x in precip.lon.values if x>=180.]
    lons_west=[x for x in precip.lon.values if x<180.]
    p_east=precip.sel(lon=lons_east)
    p_west=precip.sel(lon=lons_west)
    p_east['lon']=[x-360. for x in lons_east]
    precip=xr.concat([p_east,p_west],dim='lon')
    p_east.close()
    p_west.close()
    
    lons_east=[x for x in precip.lon.values if x>=90]
    lons_west=[x for x in precip.lon.values if x<-90]
    precip_east=precip.sel(lon=lons_east)
    precip_west=precip.sel(lon=lons_west)
    precip_east['lon']=[x-360 for x in lons_east]
    precip_west['lon']=[x+360 for x in lons_west]
    precip=xr.concat([precip_east,precip,precip_west],dim='lon')
    
    precip=precip.interp_like(field_interp)
    
    threshold_new=xr.zeros_like(precip)
    for k in range(threshold_new.time.size):
        date=pd.to_datetime(threshold_new.time.values[k])
        if date.month==2 and date.day==29:
            threshold_new1=threshold_cpc_98.sel(time=dt.datetime(1981,2,28))
            threshold_new2=threshold_cpc_98.sel(time=dt.datetime(1981,3,1))
            threshold_new3=(threshold_new1+threshold_new1)/2.
        else:
            threshold_new3=threshold_cpc_98.sel(time=dt.datetime(1981,date.month,date.day))

        threshold_new[k,:,:]=threshold_new3.values
    
    defined1=xr.ones_like(precip).sum(dim='time')
    defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
    ratio=defined/defined1
    ratio=ratio.where(ratio>0)
    
    precip_sel=precip.where(precip>=1.)
    ones_sel=xr.ones_like(precip).where(precip>=1.)
    precip_sel_95=precip_sel.where(precip>=threshold_mswep_95)
    ones_sel_95=ones_sel.where(precip>=threshold_mswep_95)
    precip_sel_99=precip_sel.where(precip>=threshold_mswep_99)
    ones_sel_99=ones_sel.where(precip>=threshold_mswep_99)
    precip_sel_98=precip_sel.where(precip>=threshold_new)
    ones_sel_98=ones_sel.where(precip>=threshold_new)
    
    precip_total_95=precip_sel_95.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_cpc>=0.9)/ratio
    freq_total_95=ones_sel_95.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_cpc>=0.9)/ratio
    intensity_sel_95=precip_sel_95.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_cpc>=0.9)
    precip_total_98=precip_sel_98.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_cpc>=0.9)/ratio
    freq_total_98=ones_sel_98.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_cpc>=0.9)/ratio
    intensity_sel_98=precip_sel_98.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_cpc>=0.9)
    precip_total_99=precip_sel_99.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_cpc>=0.9)/ratio
    freq_total_99=ones_sel_99.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_cpc>=0.9)/ratio
    intensity_sel_99=precip_sel_99.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_cpc>=0.9)
    
    if year==1990:
        total_concat_cpc_95=precip_total_95
        freq_concat_cpc_95=freq_total_95
        intensity_concat_cpc_95=intensity_sel_95
        total_concat_cpc_98=precip_total_98
        freq_concat_cpc_98=freq_total_98
        intensity_concat_cpc_98=intensity_sel_98
        total_concat_cpc_99=precip_total_99
        freq_concat_cpc_99=freq_total_99
        intensity_concat_cpc_99=intensity_sel_99

    else:
        total_concat_cpc_95=xr.concat([total_concat_cpc_95,precip_total_95],dim='time')
        freq_concat_cpc_95=xr.concat([freq_concat_cpc_95,freq_total_95],dim='time')
        intensity_concat_cpc_95=xr.concat([intensity_concat_cpc_95,intensity_sel_95],dim='time')
        total_concat_cpc_98=xr.concat([total_concat_cpc_98,precip_total_98],dim='time')
        freq_concat_cpc_98=xr.concat([freq_concat_cpc_98,freq_total_98],dim='time')
        intensity_concat_cpc_98=xr.concat([intensity_concat_cpc_98,intensity_sel_98],dim='time')
        total_concat_cpc_99=xr.concat([total_concat_cpc_99,precip_total_99],dim='time')
        freq_concat_cpc_99=xr.concat([freq_concat_cpc_99,freq_total_99],dim='time')
        intensity_concat_cpc_99=xr.concat([intensity_concat_cpc_99,intensity_sel_99],dim='time')

    ds.close()
    precip.close()
        
    ds=xr.open_dataset(dir_data+'holding/era_5_precip_'+str(year)+'.nc')
    precip=ds['precip']
    
    lons_east=[x for x in precip.lon.values if x>=90]
    lons_west=[x for x in precip.lon.values if x<-90]
    precip_east=precip.sel(lon=lons_east)
    precip_west=precip.sel(lon=lons_west)
    precip_east['lon']=[x-360 for x in lons_east]
    precip_west['lon']=[x+360 for x in lons_west]
    precip=xr.concat([precip_east,precip,precip_west],dim='lon')
    
    precip=precip.interp_like(field_interp)
    
    threshold_new=xr.zeros_like(precip)
    for k in range(threshold_new.time.size):
        date=pd.to_datetime(threshold_new.time.values[k])
        if date.month==2 and date.day==29:
            threshold_new1=threshold_era5_98.sel(time=dt.datetime(1981,2,28))
            threshold_new2=threshold_era5_98.sel(time=dt.datetime(1981,3,1))
            threshold_new3=(threshold_new1+threshold_new1)/2.
        else:
            threshold_new3=threshold_era5_98.sel(time=dt.datetime(1981,date.month,date.day))

        threshold_new[k,:,:]=threshold_new3.values
    
    defined1=xr.ones_like(precip).sum(dim='time')
    defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
    ratio=defined/defined1
    ratio=ratio.where(ratio>0)
    
    precip_sel=precip.where(precip>=1.)
    ones_sel=xr.ones_like(precip).where(precip>=1.)
    precip_sel_95=precip_sel.where(precip>=threshold_mswep_95)
    ones_sel_95=ones_sel.where(precip>=threshold_mswep_95)
    precip_sel_99=precip_sel.where(precip>=threshold_mswep_99)
    ones_sel_99=ones_sel.where(precip>=threshold_mswep_99)
    precip_sel_98=precip_sel.where(precip>=threshold_new)
    ones_sel_98=ones_sel.where(precip>=threshold_new)
    
    precip_total_95=precip_sel_95.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_era5>=0.9)/ratio
    freq_total_95=ones_sel_95.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_era5>=0.9)/ratio
    intensity_sel_95=precip_sel_95.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_era5>=0.9)
    precip_total_98=precip_sel_98.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_era5>=0.9)/ratio
    freq_total_98=ones_sel_98.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_era5>=0.9)/ratio
    intensity_sel_98=precip_sel_98.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_era5>=0.9)
    precip_total_99=precip_sel_99.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_era5>=0.9)/ratio
    freq_total_99=ones_sel_99.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_era5>=0.9)/ratio
    intensity_sel_99=precip_sel_99.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_era5>=0.9)
    
    if year==1990:
        total_concat_era5_95=precip_total_95
        freq_concat_era5_95=freq_total_95
        intensity_concat_era5_95=intensity_sel_95
        total_concat_era5_98=precip_total_98
        freq_concat_era5_98=freq_total_98
        intensity_concat_era5_98=intensity_sel_98
        total_concat_era5_99=precip_total_99
        freq_concat_era5_99=freq_total_99
        intensity_concat_era5_99=intensity_sel_99

    else:
        total_concat_era5_95=xr.concat([total_concat_era5_95,precip_total_95],dim='time')
        freq_concat_era5_95=xr.concat([freq_concat_era5_95,freq_total_95],dim='time')
        intensity_concat_era5_95=xr.concat([intensity_concat_era5_95,intensity_sel_95],dim='time')
        total_concat_era5_98=xr.concat([total_concat_era5_98,precip_total_98],dim='time')
        freq_concat_era5_98=xr.concat([freq_concat_era5_98,freq_total_98],dim='time')
        intensity_concat_era5_98=xr.concat([intensity_concat_era5_98,intensity_sel_98],dim='time')
        total_concat_era5_99=xr.concat([total_concat_era5_99,precip_total_99],dim='time')
        freq_concat_era5_99=xr.concat([freq_concat_era5_99,freq_total_99],dim='time')
        intensity_concat_era5_99=xr.concat([intensity_concat_era5_99,intensity_sel_99],dim='time')
        
    ds.close()
    precip.close()
    
    ds=xr.open_dataset(dir2+'jra55_precip_'+str(year)+'.nc')
    precip=ds['precip']
    
    lons_east=[x for x in precip.lon.values if x>=90]
    lons_west=[x for x in precip.lon.values if x<-90]
    precip_east=precip.sel(lon=lons_east)
    precip_west=precip.sel(lon=lons_west)
    precip_east['lon']=[x-360 for x in lons_east]
    precip_west['lon']=[x+360 for x in lons_west]
    precip=xr.concat([precip_east,precip,precip_west],dim='lon')
    
    precip=precip.interp_like(field_interp)
    
    threshold_new=xr.zeros_like(precip)
    for k in range(threshold_new.time.size):
        date=pd.to_datetime(threshold_new.time.values[k])
        if date.month==2 and date.day==29:
            threshold_new1=threshold_jra55_98.sel(time=dt.datetime(1981,2,28))
            threshold_new2=threshold_jra55_98.sel(time=dt.datetime(1981,3,1))
            threshold_new3=(threshold_new1+threshold_new1)/2.
        else:
            threshold_new3=threshold_jra55_98.sel(time=dt.datetime(1981,date.month,date.day))

        threshold_new[k,:,:]=threshold_new3.values
    
    defined1=xr.ones_like(precip).sum(dim='time')
    defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
    ratio=defined/defined1
    ratio=ratio.where(ratio>0)
    
    precip_sel=precip.where(precip>=1.)
    ones_sel=xr.ones_like(precip).where(precip>=1.)
    precip_sel_95=precip_sel.where(precip>=threshold_mswep_95)
    ones_sel_95=ones_sel.where(precip>=threshold_mswep_95)
    precip_sel_99=precip_sel.where(precip>=threshold_mswep_99)
    ones_sel_99=ones_sel.where(precip>=threshold_mswep_99)
    precip_sel_98=precip_sel.where(precip>=threshold_new)
    ones_sel_98=ones_sel.where(precip>=threshold_new)
    
    precip_total_95=precip_sel_95.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_jra55>=0.9)/ratio
    freq_total_95=ones_sel_95.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_jra55>=0.9)/ratio
    intensity_sel_95=precip_sel_95.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_jra55>=0.9)
    precip_total_98=precip_sel_98.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_jra55>=0.9)/ratio
    freq_total_98=ones_sel_98.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_jra55>=0.9)/ratio
    intensity_sel_98=precip_sel_98.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_jra55>=0.9)
    precip_total_99=precip_sel_99.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_jra55>=0.9)/ratio
    freq_total_99=ones_sel_99.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_jra55>=0.9)/ratio
    intensity_sel_99=precip_sel_99.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_jra55>=0.9)
    
    if year==1990:
        total_concat_jra55_95=precip_total_95
        freq_concat_jra55_95=freq_total_95
        intensity_concat_jra55_95=intensity_sel_95
        total_concat_jra55_98=precip_total_98
        freq_concat_jra55_98=freq_total_98
        intensity_concat_jra55_98=intensity_sel_98
        total_concat_jra55_99=precip_total_99
        freq_concat_jra55_99=freq_total_99
        intensity_concat_jra55_99=intensity_sel_99

    else:
        total_concat_jra55_95=xr.concat([total_concat_jra55_95,precip_total_95],dim='time')
        freq_concat_jra55_95=xr.concat([freq_concat_jra55_95,freq_total_95],dim='time')
        intensity_concat_jra55_95=xr.concat([intensity_concat_jra55_95,intensity_sel_95],dim='time')
        total_concat_jra55_98=xr.concat([total_concat_jra55_98,precip_total_98],dim='time')
        freq_concat_jra55_98=xr.concat([freq_concat_jra55_98,freq_total_98],dim='time')
        intensity_concat_jra55_98=xr.concat([intensity_concat_jra55_98,intensity_sel_98],dim='time')
        total_concat_jra55_99=xr.concat([total_concat_jra55_99,precip_total_99],dim='time')
        freq_concat_jra55_99=xr.concat([freq_concat_jra55_99,freq_total_99],dim='time')
        intensity_concat_jra55_99=xr.concat([intensity_concat_jra55_99,intensity_sel_99],dim='time')
        
    ds.close()
    precip.close()
    
    ds=xr.open_dataset(dir_data+'holding/merra2_precip_'+str(year)+'.nc')
    precip=ds['precip']
    
    lons_east=[x for x in precip.lon.values if x>=90]
    lons_west=[x for x in precip.lon.values if x<-90]
    precip_east=precip.sel(lon=lons_east)
    precip_west=precip.sel(lon=lons_west)
    precip_east['lon']=[x-360 for x in lons_east]
    precip_west['lon']=[x+360 for x in lons_west]
    precip=xr.concat([precip_east,precip,precip_west],dim='lon')
    
    precip=precip.interp_like(field_interp)
    
    threshold_new=xr.zeros_like(precip)
    for k in range(threshold_new.time.size):
        date=pd.to_datetime(threshold_new.time.values[k])
        if date.month==2 and date.day==29:
            threshold_new1=threshold_merra2_98.sel(time=dt.datetime(1981,2,28))
            threshold_new2=threshold_merra2_98.sel(time=dt.datetime(1981,3,1))
            threshold_new3=(threshold_new1+threshold_new1)/2.
        else:
            threshold_new3=threshold_merra2_98.sel(time=dt.datetime(1981,date.month,date.day))

        threshold_new[k,:,:]=threshold_new3.values
    
    defined1=xr.ones_like(precip).sum(dim='time')
    defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
    ratio=defined/defined1
    ratio=ratio.where(ratio>0)
    
    precip_sel=precip.where(precip>=1.)
    ones_sel=xr.ones_like(precip).where(precip>=1.)
    precip_sel_95=precip_sel.where(precip>=threshold_mswep_95)
    ones_sel_95=ones_sel.where(precip>=threshold_mswep_95)
    precip_sel_99=precip_sel.where(precip>=threshold_mswep_99)
    ones_sel_99=ones_sel.where(precip>=threshold_mswep_99)
    precip_sel_98=precip_sel.where(precip>=threshold_new)
    ones_sel_98=ones_sel.where(precip>=threshold_new)
    
    precip_total_95=precip_sel_95.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_merra2>=0.9)/ratio
    freq_total_95=ones_sel_95.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_merra2>=0.9)/ratio
    intensity_sel_95=precip_sel_95.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_merra2>=0.9)
    precip_total_98=precip_sel_98.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_merra2>=0.9)/ratio
    freq_total_98=ones_sel_98.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_merra2>=0.9)/ratio
    intensity_sel_98=precip_sel_98.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_merra2>=0.9)
    precip_total_99=precip_sel_99.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_merra2>=0.9)/ratio
    freq_total_99=ones_sel_99.sum(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_merra2>=0.9)/ratio
    intensity_sel_99=precip_sel_99.mean(dim='time',skipna=True).where(ratio>=0.9).where(ratio_all_merra2>=0.9)
    
    if year==1990:
        total_concat_merra2_95=precip_total_95
        freq_concat_merra2_95=freq_total_95
        intensity_concat_merra2_95=intensity_sel_95
        total_concat_merra2_98=precip_total_98
        freq_concat_merra2_98=freq_total_98
        intensity_concat_merra2_98=intensity_sel_98
        total_concat_merra2_99=precip_total_99
        freq_concat_merra2_99=freq_total_99
        intensity_concat_merra2_99=intensity_sel_99

    else:
        total_concat_merra2_95=xr.concat([total_concat_merra2_95,precip_total_95],dim='time')
        freq_concat_merra2_95=xr.concat([freq_concat_merra2_95,freq_total_95],dim='time')
        intensity_concat_merra2_95=xr.concat([intensity_concat_merra2_95,intensity_sel_95],dim='time')
        total_concat_merra2_98=xr.concat([total_concat_merra2_98,precip_total_98],dim='time')
        freq_concat_merra2_98=xr.concat([freq_concat_merra2_98,freq_total_98],dim='time')
        intensity_concat_merra2_98=xr.concat([intensity_concat_merra2_98,intensity_sel_98],dim='time')
        total_concat_merra2_99=xr.concat([total_concat_merra2_99,precip_total_99],dim='time')
        freq_concat_merra2_99=xr.concat([freq_concat_merra2_99,freq_total_99],dim='time')
        intensity_concat_merra2_99=xr.concat([intensity_concat_merra2_99,intensity_sel_99],dim='time')
    
    ds.close()
    precip.close()

In [ ]:
dk=xr.Dataset()
dk['total_95_mswep']=(('time_mswep','lat','lon'),total_concat_mswep_95.values)
dk['total_98_mswep']=(('time_mswep','lat','lon'),total_concat_mswep_98.values)
dk['total_99_mswep']=(('time_mswep','lat','lon'),total_concat_mswep_99.values)

dk['freq_95_mswep']=(('time_mswep','lat','lon'),freq_concat_mswep_95.values)
dk['freq_98_mswep']=(('time_mswep','lat','lon'),freq_concat_mswep_98.values)
dk['freq_99_mswep']=(('time_mswep','lat','lon'),freq_concat_mswep_99.values)

dk['intensity_95_mswep']=(('time_mswep','lat','lon'),intensity_concat_mswep_95.values)
dk['intensity_98_mswep']=(('time_mswep','lat','lon'),intensity_concat_mswep_98.values)
dk['intensity_99_mswep']=(('time_mswep','lat','lon'),intensity_concat_mswep_99.values)


dk['total_95_persiann']=(('time_persiann','lat','lon'),total_concat_persiann_95.values)
dk['total_98_persiann']=(('time_persiann','lat','lon'),total_concat_persiann_98.values)
dk['total_99_persiann']=(('time_persiann','lat','lon'),total_concat_persiann_99.values)

dk['freq_95_persiann']=(('time_persiann','lat','lon'),freq_concat_persiann_95.values)
dk['freq_98_persiann']=(('time_persiann','lat','lon'),freq_concat_persiann_98.values)
dk['freq_99_persiann']=(('time_persiann','lat','lon'),freq_concat_persiann_99.values)

dk['intensity_95_persiann']=(('time_persiann','lat','lon'),intensity_concat_persiann_95.values)
dk['intensity_98_persiann']=(('time_persiann','lat','lon'),intensity_concat_persiann_98.values)
dk['intensity_99_persiann']=(('time_persiann','lat','lon'),intensity_concat_persiann_99.values)


dk['total_95_regen']=(('time_regen','lat','lon'),total_concat_regen_95.values)
dk['total_98_regen']=(('time_regen','lat','lon'),total_concat_regen_98.values)
dk['total_99_regen']=(('time_regen','lat','lon'),total_concat_regen_99.values)

dk['freq_95_regen']=(('time_regen','lat','lon'),freq_concat_regen_95.values)
dk['freq_98_regen']=(('time_regen','lat','lon'),freq_concat_regen_98.values)
dk['freq_99_regen']=(('time_regen','lat','lon'),freq_concat_regen_99.values)

dk['intensity_95_regen']=(('time_regen','lat','lon'),intensity_concat_regen_95.values)
dk['intensity_98_regen']=(('time_regen','lat','lon'),intensity_concat_regen_98.values)
dk['intensity_99_regen']=(('time_regen','lat','lon'),intensity_concat_regen_99.values)


dk['total_95_gpcc']=(('time_gpcc','lat','lon'),total_concat_gpcc_95.values)
dk['total_98_gpcc']=(('time_gpcc','lat','lon'),total_concat_gpcc_98.values)
dk['total_99_gpcc']=(('time_gpcc','lat','lon'),total_concat_gpcc_99.values)

dk['freq_95_gpcc']=(('time_gpcc','lat','lon'),freq_concat_gpcc_95.values)
dk['freq_98_gpcc']=(('time_gpcc','lat','lon'),freq_concat_gpcc_98.values)
dk['freq_99_gpcc']=(('time_gpcc','lat','lon'),freq_concat_gpcc_99.values)

dk['intensity_95_gpcc']=(('time_gpcc','lat','lon'),intensity_concat_gpcc_95.values)
dk['intensity_98_gpcc']=(('time_gpcc','lat','lon'),intensity_concat_gpcc_98.values)
dk['intensity_99_gpcc']=(('time_gpcc','lat','lon'),intensity_concat_gpcc_99.values)


dk['total_95_cpc']=(('time','lat','lon'),total_concat_cpc_95.values)
dk['total_98_cpc']=(('time','lat','lon'),total_concat_cpc_98.values)
dk['total_99_cpc']=(('time','lat','lon'),total_concat_cpc_99.values)

dk['freq_95_cpc']=(('time','lat','lon'),freq_concat_cpc_95.values)
dk['freq_98_cpc']=(('time','lat','lon'),freq_concat_cpc_98.values)
dk['freq_99_cpc']=(('time','lat','lon'),freq_concat_cpc_99.values)

dk['intensity_95_cpc']=(('time','lat','lon'),intensity_concat_cpc_95.values)
dk['intensity_98_cpc']=(('time','lat','lon'),intensity_concat_cpc_98.values)
dk['intensity_99_cpc']=(('time','lat','lon'),intensity_concat_cpc_99.values)


dk['total_95_era5']=(('time','lat','lon'),total_concat_era5_95.values)
dk['total_98_era5']=(('time','lat','lon'),total_concat_era5_98.values)
dk['total_99_era5']=(('time','lat','lon'),total_concat_era5_99.values)

dk['freq_95_era5']=(('time','lat','lon'),freq_concat_era5_95.values)
dk['freq_98_era5']=(('time','lat','lon'),freq_concat_era5_98.values)
dk['freq_99_era5']=(('time','lat','lon'),freq_concat_era5_99.values)

dk['intensity_95_era5']=(('time','lat','lon'),intensity_concat_era5_95.values)
dk['intensity_98_era5']=(('time','lat','lon'),intensity_concat_era5_98.values)
dk['intensity_99_era5']=(('time','lat','lon'),intensity_concat_era5_99.values)


dk['total_95_jra55']=(('time','lat','lon'),total_concat_jra55_95.values)
dk['total_98_jra55']=(('time','lat','lon'),total_concat_jra55_98.values)
dk['total_99_jra55']=(('time','lat','lon'),total_concat_jra55_99.values)

dk['freq_95_jra55']=(('time','lat','lon'),freq_concat_jra55_95.values)
dk['freq_98_jra55']=(('time','lat','lon'),freq_concat_jra55_98.values)
dk['freq_99_jra55']=(('time','lat','lon'),freq_concat_jra55_99.values)

dk['intensity_95_jra55']=(('time','lat','lon'),intensity_concat_jra55_95.values)
dk['intensity_98_jra55']=(('time','lat','lon'),intensity_concat_jra55_98.values)
dk['intensity_99_jra55']=(('time','lat','lon'),intensity_concat_jra55_99.values)


dk['total_95_merra2']=(('time','lat','lon'),total_concat_merra2_95.values)
dk['total_98_merra2']=(('time','lat','lon'),total_concat_merra2_98.values)
dk['total_99_merra2']=(('time','lat','lon'),total_concat_merra2_99.values)

dk['freq_95_merra2']=(('time','lat','lon'),freq_concat_merra2_95.values)
dk['freq_98_merra2']=(('time','lat','lon'),freq_concat_merra2_98.values)
dk['freq_99_merra2']=(('time','lat','lon'),freq_concat_merra2_99.values)

dk['intensity_95_merra2']=(('time','lat','lon'),intensity_concat_merra2_95.values)
dk['intensity_98_merra2']=(('time','lat','lon'),intensity_concat_merra2_98.values)
dk['intensity_99_merra2']=(('time','lat','lon'),intensity_concat_merra2_99.values)

dk.coords['time']=np.arange(1990,2023,1)
dk.coords['time_mswep']=np.arange(1990,2021,1)
dk.coords['time_gpcc']=np.arange(1990,2021,1)
dk.coords['time_regen']=np.arange(1990,2017,1)
dk.coords['time_persiann']=np.arange(1990,2023,1)
dk.coords['lat']=total_concat_mswep_95.lat.values
dk.coords['lon']=total_concat_mswep_95.lon.values
dk['lat'].attrs["units"]='degrees_north'
dk['lon'].attrs["units"]='degrees_east'
dk['time'].attrs["units"]='years'
dk['time_mswep'].attrs["units"]='years'
dk['time_gpcc'].attrs["units"]='years'
dk['time_persiann'].attrs["units"]='years'
dk['time_regen'].attrs["units"]='years'

try:
    os.remove(dir_data+'precip_geomeans_interpolated_1990.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'precip_geomeans_interpolated_1990.nc',mode='w',format='NETCDF4')

In [ ]:
ds=xr.open_dataset(dir_data+'precip_geomeans_interpolated_1990.nc')

total_95_mswep=ds['total_95_mswep']
total_98_mswep=ds['total_98_mswep']
total_99_mswep=ds['total_99_mswep']

total_95_persiann=ds['total_95_persiann']
total_98_persiann=ds['total_98_persiann']
total_99_persiann=ds['total_99_persiann']

total_95_regen=ds['total_95_regen']
total_98_regen=ds['total_98_regen']
total_99_regen=ds['total_99_regen']

total_95_gpcc=ds['total_95_gpcc']
total_98_gpcc=ds['total_98_gpcc']
total_99_gpcc=ds['total_99_gpcc']

total_95_cpc=ds['total_95_cpc']
total_98_cpc=ds['total_98_cpc']
total_99_cpc=ds['total_99_cpc']

total_95_era5=ds['total_95_era5']
total_98_era5=ds['total_98_era5']
total_99_era5=ds['total_99_era5']

total_95_jra55=ds['total_95_jra55']
total_98_jra55=ds['total_98_jra55']
total_99_jra55=ds['total_99_jra55']

total_95_merra2=ds['total_95_merra2']
total_98_merra2=ds['total_98_merra2']
total_99_merra2=ds['total_99_merra2']

In [ ]:
test=total_99_persiann.sum(dim='time_persiann',skipna=True)
print(test.values)

fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(test.lon,test.lat,test,vmin=0,vmax=5000,cmap=plt.cm.RdYlGn_r)
ax.coastlines(resolution='10m')
plt.show()

In [ ]:
slopes_array_95_mswep=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_95_mswep=xr.zeros_like(total_95_mswep[0,:,:])
slopes_array_98_mswep=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_98_mswep=xr.zeros_like(total_95_mswep[0,:,:])
slopes_array_99_mswep=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_99_mswep=xr.zeros_like(total_95_mswep[0,:,:])

slopes_array_95_persiann=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_95_persiann=xr.zeros_like(total_95_mswep[0,:,:])
slopes_array_98_persiann=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_98_persiann=xr.zeros_like(total_95_mswep[0,:,:])
slopes_array_99_persiann=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_99_persiann=xr.zeros_like(total_95_mswep[0,:,:])

slopes_array_95_regen=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_95_regen=xr.zeros_like(total_95_mswep[0,:,:])
slopes_array_98_regen=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_98_regen=xr.zeros_like(total_95_mswep[0,:,:])
slopes_array_99_regen=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_99_regen=xr.zeros_like(total_95_mswep[0,:,:])

slopes_array_95_gpcc=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_95_gpcc=xr.zeros_like(total_95_mswep[0,:,:])
slopes_array_98_gpcc=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_98_gpcc=xr.zeros_like(total_95_mswep[0,:,:])
slopes_array_99_gpcc=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_99_gpcc=xr.zeros_like(total_95_mswep[0,:,:])

slopes_array_95_cpc=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_95_cpc=xr.zeros_like(total_95_mswep[0,:,:])
slopes_array_98_cpc=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_98_cpc=xr.zeros_like(total_95_mswep[0,:,:])
slopes_array_99_cpc=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_99_cpc=xr.zeros_like(total_95_mswep[0,:,:])

slopes_array_95_era5=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_95_era5=xr.zeros_like(total_95_mswep[0,:,:])
slopes_array_98_era5=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_98_era5=xr.zeros_like(total_95_mswep[0,:,:])
slopes_array_99_era5=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_99_era5=xr.zeros_like(total_95_mswep[0,:,:])

slopes_array_95_jra55=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_95_jra55=xr.zeros_like(total_95_mswep[0,:,:])
slopes_array_98_jra55=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_98_jra55=xr.zeros_like(total_95_mswep[0,:,:])
slopes_array_99_jra55=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_99_jra55=xr.zeros_like(total_95_mswep[0,:,:])

slopes_array_95_merra2=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_95_merra2=xr.zeros_like(total_95_mswep[0,:,:])
slopes_array_98_merra2=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_98_merra2=xr.zeros_like(total_95_mswep[0,:,:])
slopes_array_99_merra2=xr.zeros_like(total_95_mswep[0,:,:])
sigs_array_99_merra2=xr.zeros_like(total_95_mswep[0,:,:])
for j in range(slopes_array_95_mswep.lat.size):
    print(j)
    for k in range(slopes_array_95_mswep.lon.size):
        yrs=np.arange(1990,2021,1)
        time_series=total_95_mswep[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_95_mswep[j,k]=r[0]
            sigs_array_95_mswep[j,k]=r[3]
        else:
            slopes_array_95_mswep[j,k]=np.nan
            sigs_array_95_mswep[j,k]=np.nan
        
        time_series=total_98_mswep[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_98_mswep[j,k]=r[0]
            sigs_array_98_mswep[j,k]=r[3]
        else:
            slopes_array_98_mswep[j,k]=np.nan
            sigs_array_98_mswep[j,k]=np.nan
        
        time_series=total_99_mswep[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_99_mswep[j,k]=r[0]
            sigs_array_99_mswep[j,k]=r[3]
        else:
            slopes_array_99_mswep[j,k]=np.nan
            sigs_array_99_mswep[j,k]=np.nan
        
        
        yrs=np.arange(1990,2023,1)
        time_series=total_95_persiann[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_95_persiann[j,k]=r[0]
            sigs_array_95_persiann[j,k]=r[3]
        else:
            slopes_array_95_persiann[j,k]=np.nan
            sigs_array_95_persiann[j,k]=np.nan
        
        time_series=total_98_persiann[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_98_persiann[j,k]=r[0]
            sigs_array_98_persiann[j,k]=r[3]
        else:
            slopes_array_98_persiann[j,k]=np.nan
            sigs_array_98_persiann[j,k]=np.nan
        
        time_series=total_99_persiann[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_99_persiann[j,k]=r[0]
            sigs_array_99_persiann[j,k]=r[3]
        else:
            slopes_array_99_persiann[j,k]=np.nan
            sigs_array_99_persiann[j,k]=np.nan

        
        yrs=np.arange(1990,2017,1)
        time_series=total_95_regen[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_95_regen[j,k]=r[0]
            sigs_array_95_regen[j,k]=r[3]
        else:
            slopes_array_95_regen[j,k]=np.nan
            sigs_array_95_regen[j,k]=np.nan
        
        time_series=total_98_regen[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_98_regen[j,k]=r[0]
            sigs_array_98_regen[j,k]=r[3]
        else:
            slopes_array_98_regen[j,k]=np.nan
            sigs_array_98_regen[j,k]=np.nan
        
        time_series=total_99_regen[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_99_regen[j,k]=r[0]
            sigs_array_99_regen[j,k]=r[3]
        else:
            slopes_array_99_regen[j,k]=np.nan
            sigs_array_99_regen[j,k]=np.nan
        
        
        yrs=np.arange(1990,2021,1)
        time_series=total_95_gpcc[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_95_gpcc[j,k]=r[0]
            sigs_array_95_gpcc[j,k]=r[3]
        else:
            slopes_array_95_gpcc[j,k]=np.nan
            sigs_array_95_gpcc[j,k]=np.nan
        
        time_series=total_98_gpcc[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_98_gpcc[j,k]=r[0]
            sigs_array_98_gpcc[j,k]=r[3]
        else:
            slopes_array_98_gpcc[j,k]=np.nan
            sigs_array_98_gpcc[j,k]=np.nan
        
        time_series=total_99_gpcc[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_99_gpcc[j,k]=r[0]
            sigs_array_99_gpcc[j,k]=r[3]
        else:
            slopes_array_99_gpcc[j,k]=np.nan
            sigs_array_99_gpcc[j,k]=np.nan

        
        yrs=np.arange(1990,2023,1)
        time_series=total_95_cpc[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_95_cpc[j,k]=r[0]
            sigs_array_95_cpc[j,k]=r[3]
        else:
            slopes_array_95_cpc[j,k]=np.nan
            sigs_array_95_cpc[j,k]=np.nan
        
        time_series=total_98_cpc[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_98_cpc[j,k]=r[0]
            sigs_array_98_cpc[j,k]=r[3]
        else:
            slopes_array_98_cpc[j,k]=np.nan
            sigs_array_98_cpc[j,k]=np.nan
        
        time_series=total_99_cpc[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_99_cpc[j,k]=r[0]
            sigs_array_99_cpc[j,k]=r[3]
        else:
            slopes_array_99_cpc[j,k]=np.nan
            sigs_array_99_cpc[j,k]=np.nan
        
        
        yrs=np.arange(1990,2023,1)
        time_series=total_95_era5[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_95_era5[j,k]=r[0]
            sigs_array_95_era5[j,k]=r[3]
        else:
            slopes_array_95_era5[j,k]=np.nan
            sigs_array_95_era5[j,k]=np.nan
        
        time_series=total_98_era5[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_98_era5[j,k]=r[0]
            sigs_array_98_era5[j,k]=r[3]
        else:
            slopes_array_98_era5[j,k]=np.nan
            sigs_array_98_era5[j,k]=np.nan
        
        time_series=total_99_era5[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_99_era5[j,k]=r[0]
            sigs_array_99_era5[j,k]=r[3]
        else:
            slopes_array_99_era5[j,k]=np.nan
            sigs_array_99_era5[j,k]=np.nan
        
        
        yrs=np.arange(1990,2023,1)
        time_series=total_95_jra55[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_95_jra55[j,k]=r[0]
            sigs_array_95_jra55[j,k]=r[3]
        else:
            slopes_array_95_jra55[j,k]=np.nan
            sigs_array_95_jra55[j,k]=np.nan
        
        time_series=total_98_jra55[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_98_jra55[j,k]=r[0]
            sigs_array_98_jra55[j,k]=r[3]
        else:
            slopes_array_98_jra55[j,k]=np.nan
            sigs_array_98_jra55[j,k]=np.nan
        
        time_series=total_99_jra55[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_99_jra55[j,k]=r[0]
            sigs_array_99_jra55[j,k]=r[3]
        else:
            slopes_array_99_jra55[j,k]=np.nan
            sigs_array_99_jra55[j,k]=np.nan
        
        
        yrs=np.arange(1990,2023,1)
        time_series=total_95_merra2[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_95_merra2[j,k]=r[0]
            sigs_array_95_merra2[j,k]=r[3]
        else:
            slopes_array_95_merra2[j,k]=np.nan
            sigs_array_95_merra2[j,k]=np.nan
        
        time_series=total_98_merra2[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_98_merra2[j,k]=r[0]
            sigs_array_98_merra2[j,k]=r[3]
        else:
            slopes_array_98_merra2[j,k]=np.nan
            sigs_array_98_merra2[j,k]=np.nan
        
        time_series=total_99_merra2[:,j,k].values.tolist()
        zipped_years=list(zip(yrs,time_series))
        select_years=[x[0] for x in zipped_years if x[1]>=0]
        select_values=[x[1] for x in zipped_years if x[1]>=0]
        if len(select_years)>=10:
            r=st.linregress(select_years,select_values)
            slopes_array_99_merra2[j,k]=r[0]
            sigs_array_99_merra2[j,k]=r[3]
        else:
            slopes_array_99_merra2[j,k]=np.nan
            sigs_array_99_merra2[j,k]=np.nan

In [ ]:
p_values=sigs_array_95_mswep.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
a=0.05#/sum_factor
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_95_mswep=max(pass_list)
print(p_final_95_mswep)

p_values=sigs_array_98_mswep.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_98_mswep=max(pass_list)
print(p_final_98_mswep)

p_values=sigs_array_99_mswep.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_99_mswep=max(pass_list)
print(p_final_99_mswep)

In [ ]:
field=slopes_array_95_mswep
field1=sigs_array_95_mswep
mean=total_95_mswep.mean(dim='time_mswep',skipna=True)

field=field/mean*100.

from matplotlib.colors import ListedColormap
cmap1=plt.cm.seismic(np.linspace(0.2,0.8,128))
cmap_new=ListedColormap(cmap1)

fig=plt.figure(figsize=(24,7))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.pcolormesh(field.lon,field.lat,field,vmin=-1.5,vmax=1.5,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=15)

#X=6
X=2
for j in range(int(field1.lat.size/X)):
    for k in range(int(field1.lon.size/X)):
        if field1[X*j,X*k].values.tolist()<=p_final_95_mswep:
            ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='yellow',markersize=3.5)
            #ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='yellow',markersize=3.5)
        #elif field1[X*j,X*k].values.tolist()<=0.05:
        #    ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='w',markersize=1.75)#1.5

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=26) 
cbar.set_label('% yr$^{-1}$',fontsize=27,labelpad=5)
ax.coastlines(resolution='10m',linewidth=2)
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':24,'color':'k'}
g1.ylabel_style={'size':24,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
ax.set_ylim(-60,70)
#ax.set_ylim(-89.5,89.5)
           
ax.set_title(r'$\bf{1980-2020}$'+' '+r'$\bf{interpolated}$',fontsize=42,pad=0)
#ax.set_title(r'$\bf{1979-2020}$',fontsize=42,pad=0)
#ax.set_title(r'$\bf{R1mm}$',fontsize=46,pad=0)
#ax.set_title(r'$\bf{R98p_{all}}$',fontsize=40,pad=0)
#ax.set_title(r'$\bf{R1mm}$',fontsize=40,pad=0)
plt.show()

In [ ]:
fig.savefig(dir_data+'precip_trends_interpolated_demo_mswep.png')

In [ ]:
p_values=sigs_array_95_persiann.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
print(N)
a=0.05#/sum_factor
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_95_persiann=max(pass_list)
print(p_final_95_persiann)

p_values=sigs_array_98_persiann.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_98_persiann=max(pass_list)
print(p_final_98_persiann)

p_values=sigs_array_99_persiann.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_99_persiann=max(pass_list)
print(p_final_99_persiann)

In [ ]:
field=slopes_array_95_persiann
field1=sigs_array_95_persiann
mean=total_95_persiann.mean(dim='time_persiann',skipna=True)

field=field/mean*100.

from matplotlib.colors import ListedColormap
cmap1=plt.cm.seismic(np.linspace(0.2,0.8,128))
cmap_new=ListedColormap(cmap1)

fig=plt.figure(figsize=(24,7))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.pcolormesh(field.lon,field.lat,field,vmin=-1.5,vmax=1.5,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=15)

#X=6
X=2
for j in range(int(field1.lat.size/X)):
    for k in range(int(field1.lon.size/X)):
        if field1[X*j,X*k].values.tolist()<=p_final_95_persiann:
            ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='yellow',markersize=3.5)
            #ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='yellow',markersize=3.5)
        #elif field1[X*j,X*k].values.tolist()<=0.05:
        #    ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='w',markersize=1.75)#1.5

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=26) 
cbar.set_label('% yr$^{-1}$',fontsize=27,labelpad=5)
ax.coastlines(resolution='10m',linewidth=2)
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':24,'color':'k'}
g1.ylabel_style={'size':24,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
ax.set_ylim(-60,70)
#ax.set_ylim(-89.5,89.5)
           
#ax.set_title(r'$\bf{1983-2022}$'+' '+r'$\bf{interpolated}$',fontsize=42,pad=0)
#ax.set_title(r'$\bf{1979-2020}$',fontsize=42,pad=0)
ax.set_title(r'$\bf{1983-2022}$',fontsize=42,pad=0)
#ax.set_title(r'$\bf{R1mm}$',fontsize=46,pad=0)
#ax.set_title(r'$\bf{R98p_{all}}$',fontsize=40,pad=0)
#ax.set_title(r'$\bf{R1mm}$',fontsize=40,pad=0)
plt.show()

In [ ]:
fig.savefig(dir_data+'precip_trends_interpolated_demo_persianna.png')

In [ ]:
p_values=sigs_array_95_gpcc.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
print(N)
a=0.05#/sum_factor
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_95_gpcc=max(pass_list)
print(p_final_95_gpcc)

p_values=sigs_array_98_gpcc.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_98_gpcc=max(pass_list)
print(p_final_98_gpcc)

p_values=sigs_array_99_gpcc.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_99_gpcc=max(pass_list)
print(p_final_99_gpcc)

In [ ]:
field=slopes_array_95_gpcc
field1=sigs_array_95_gpcc
mean=total_95_gpcc.mean(dim='time_gpcc',skipna=True)

field=field/mean*100.

from matplotlib.colors import ListedColormap
cmap1=plt.cm.seismic(np.linspace(0.2,0.8,128))
cmap_new=ListedColormap(cmap1)

fig=plt.figure(figsize=(24,7))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.pcolormesh(field.lon,field.lat,field,vmin=-1.5,vmax=1.5,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=15)

#X=6
X=2
for j in range(int(field1.lat.size/X)):
    for k in range(int(field1.lon.size/X)):
        if field1[X*j,X*k].values.tolist()<=p_final_95_gpcc:
            ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='yellow',markersize=3.5)
            #ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='yellow',markersize=3.5)
        #elif field1[X*j,X*k].values.tolist()<=0.05:
        #    ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='w',markersize=1.75)#1.5

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=26) 
cbar.set_label('% yr$^{-1}$',fontsize=27,labelpad=5)
ax.coastlines(resolution='10m',linewidth=2)
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':24,'color':'k'}
g1.ylabel_style={'size':24,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
ax.set_ylim(-60,70)
#ax.set_ylim(-89.5,89.5)
           
#ax.set_title(r'$\bf{1982-2020}$'+' '+r'$\bf{interpolated}$',fontsize=42,pad=0)
ax.set_title(r'$\bf{1982-2020}$',fontsize=42,pad=0)
#ax.set_title(r'$\bf{R1mm}$',fontsize=46,pad=0)
#ax.set_title(r'$\bf{R98p_{all}}$',fontsize=40,pad=0)
#ax.set_title(r'$\bf{R1mm}$',fontsize=40,pad=0)
plt.show()

In [ ]:
fig.savefig(dir_data+'precip_trends_interpolated_demo_gpcca.png')

In [ ]:
p_values=sigs_array_95_merra2.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
print(N)
a=0.05#/sum_factor
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_95_merra2=max(pass_list)
print(p_final_95_merra2)

p_values=sigs_array_98_merra2.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_98_merra2=max(pass_list)
print(p_final_98_merra2)

p_values=sigs_array_99_merra2.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_99_merra2=max(pass_list)
print(p_final_99_merra2)

In [ ]:
field=slopes_array_95_merra2
field1=sigs_array_95_merra2
mean=total_95_merra2.mean(dim='time',skipna=True)

field=field/mean*100.

from matplotlib.colors import ListedColormap
cmap1=plt.cm.seismic(np.linspace(0.2,0.8,128))
cmap_new=ListedColormap(cmap1)

fig=plt.figure(figsize=(24,7))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.pcolormesh(field.lon,field.lat,field,vmin=-1.5,vmax=1.5,cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,extend='both',aspect=15)

#X=6
X=2
for j in range(int(field1.lat.size/X)):
    for k in range(int(field1.lon.size/X)):
        if field1[X*j,X*k].values.tolist()<=p_final_95_merra2:
            ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='yellow',markersize=3.5)
            #ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='yellow',markersize=3.5)
        #elif field1[X*j,X*k].values.tolist()<=0.05:
        #    ax.plot(field.lon.values[X*k],field.lat.values[X*j],marker='o',color='w',markersize=1.75)#1.5

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=26) 
cbar.set_label('% yr$^{-1}$',fontsize=27,labelpad=5)
ax.coastlines(resolution='10m',linewidth=2)
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':24,'color':'k'}
g1.ylabel_style={'size':24,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
ax.set_ylim(-60,70)
#ax.set_ylim(-89.5,89.5)
           
#ax.set_title(r'$\bf{1980-2022}$'+' '+r'$\bf{interpolated}$',fontsize=42,pad=0)
ax.set_title(r'$\bf{1980-2022}$',fontsize=42,pad=0)
#ax.set_title(r'$\bf{R1mm}$',fontsize=46,pad=0)
#ax.set_title(r'$\bf{R98p_{all}}$',fontsize=40,pad=0)
#ax.set_title(r'$\bf{R1mm}$',fontsize=40,pad=0)
plt.show()

In [ ]:
fig.savefig(dir_data+'precip_trends_interpolated_demo_merra2a.png')

In [ ]:
p_values=sigs_array_95_jra55.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
a=0.05#/sum_factor
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_95_jra55=max(pass_list)
print(p_final_95_jra55)

p_values=sigs_array_98_jra55.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_98_jra55=max(pass_list)
print(p_final_98_jra55)

p_values=sigs_array_99_jra55.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_99_jra55=max(pass_list)
print(p_final_99_jra55)


p_values=sigs_array_95_era5.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
a=0.05#/sum_factor
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_95_era5=max(pass_list)
print(p_final_95_era5)

p_values=sigs_array_98_era5.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_98_era5=max(pass_list)
print(p_final_98_era5)

p_values=sigs_array_99_era5.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_99_era5=max(pass_list)
print(p_final_99_era5)


p_values=sigs_array_95_regen.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
a=0.05#/sum_factor
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_95_regen=max(pass_list)
print(p_final_95_regen)

p_values=sigs_array_98_regen.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_98_regen=max(pass_list)
print(p_final_98_regen)

p_values=sigs_array_99_regen.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_99_regen=max(pass_list)
print(p_final_99_regen)


p_values=sigs_array_95_cpc.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
a=0.05#/sum_factor
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_95_cpc=max(pass_list)
print(p_final_95_cpc)

p_values=sigs_array_98_cpc.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_98_cpc=max(pass_list)
print(p_final_98_cpc)

p_values=sigs_array_99_cpc.values.flatten().tolist()
p_values=sorted([x for x in p_values if x>=0])
N=len(p_values)
pass_list=[]
for i in range(len(p_values)):
    index=i+1
    p_value=p_values[i]
    if p_value<=a*index/float(N):
        pass_list.append(p_value)
p_final_99_cpc=max(pass_list)
print(p_final_99_cpc)

In [ ]:
num_sets1=xr.ones_like(slopes_array_95_regen).where(slopes_array_95_regen>-1000).fillna(0)
num_sets2=xr.ones_like(slopes_array_95_cpc).where(slopes_array_95_cpc>-1000).fillna(0)
num_sets3=xr.ones_like(slopes_array_95_gpcc).where(slopes_array_95_gpcc>-1000).fillna(0)
num_sets4=xr.ones_like(slopes_array_95_persiann).where(slopes_array_95_persiann>-1000).fillna(0)
num_sets5=xr.ones_like(slopes_array_95_mswep).where(slopes_array_95_mswep>-1000).fillna(0)
num_sets6=xr.ones_like(slopes_array_95_era5).where(slopes_array_95_era5>-1000).fillna(0)
num_sets7=xr.ones_like(slopes_array_95_jra55).where(slopes_array_95_jra55>-1000).fillna(0)
num_sets8=xr.ones_like(slopes_array_95_merra2).where(slopes_array_95_merra2>-1000).fillna(0)
num_sets=num_sets1+num_sets2+num_sets3+num_sets4+num_sets5+num_sets6+num_sets7+num_sets8

field=num_sets

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>0),levels=np.arange(0,9,1),cmap=plt.cm.Reds,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,9,1))

#cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>0)*-1,levels=np.arange(-8,9,1),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=36) 
cbar.set_label('# datasets',fontsize=42,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':36,'color':'k'}
g1.ylabel_style={'size':36,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{Number}$'+' '+r'$\bf{of}$'+' '+r'$\bf{40+}$'+' '+r'$\bf{year}$'+' '+r'$\bf{datasets}$'+' ',fontsize=48,pad=5)

plt.show()

In [ ]:
test1=xr.ones_like(sigs_array_95_mswep).where(slopes_array_95_mswep>0).fillna(0)
test2=xr.ones_like(sigs_array_95_persiann).where(slopes_array_95_persiann>0).fillna(0)
test3=xr.ones_like(sigs_array_95_regen).where(slopes_array_95_regen>0).fillna(0)
test4=xr.ones_like(sigs_array_95_gpcc).where(slopes_array_95_gpcc>0).fillna(0)
test5=xr.ones_like(sigs_array_95_cpc).where(slopes_array_95_cpc>0).fillna(0)
test6=xr.ones_like(sigs_array_95_era5).where(slopes_array_95_era5>0).fillna(0)
test7=xr.ones_like(sigs_array_95_jra55).where(slopes_array_95_jra55>0).fillna(0)
test8=xr.ones_like(sigs_array_95_merra2).where(slopes_array_95_merra2>0).fillna(0)

test1a=xr.ones_like(sigs_array_95_mswep).where(slopes_array_95_mswep<0).fillna(0)
test2a=xr.ones_like(sigs_array_95_persiann).where(slopes_array_95_persiann<0).fillna(0)
test3a=xr.ones_like(sigs_array_95_regen).where(slopes_array_95_regen<0).fillna(0)
test4a=xr.ones_like(sigs_array_95_gpcc).where(slopes_array_95_gpcc<0).fillna(0)
test5a=xr.ones_like(sigs_array_95_cpc).where(slopes_array_95_cpc<0).fillna(0)
test6a=xr.ones_like(sigs_array_95_era5).where(slopes_array_95_era5<0).fillna(0)
test7a=xr.ones_like(sigs_array_95_jra55).where(slopes_array_95_jra55<0).fillna(0)
test8a=xr.ones_like(sigs_array_95_merra2).where(slopes_array_95_merra2<0).fillna(0)

test9=xr.ones_like(sigs_array_99_mswep).where(slopes_array_99_mswep>0).fillna(0)
test10=xr.ones_like(sigs_array_99_persiann).where(slopes_array_99_persiann>0).fillna(0)
test11=xr.ones_like(sigs_array_99_regen).where(slopes_array_99_regen>0).fillna(0)
test12=xr.ones_like(sigs_array_99_gpcc).where(slopes_array_99_gpcc>0).fillna(0)
test13=xr.ones_like(sigs_array_99_cpc).where(slopes_array_99_cpc>0).fillna(0)
test14=xr.ones_like(sigs_array_99_era5).where(slopes_array_99_era5>0).fillna(0)
test15=xr.ones_like(sigs_array_99_jra55).where(slopes_array_99_jra55>0).fillna(0)
test16=xr.ones_like(sigs_array_99_merra2).where(slopes_array_99_merra2>0).fillna(0)

test9a=xr.ones_like(sigs_array_99_mswep).where(slopes_array_99_mswep<0).fillna(0)
test10a=xr.ones_like(sigs_array_99_persiann).where(slopes_array_99_persiann<0).fillna(0)
test11a=xr.ones_like(sigs_array_99_regen).where(slopes_array_99_regen<0).fillna(0)
test12a=xr.ones_like(sigs_array_99_gpcc).where(slopes_array_99_gpcc<0).fillna(0)
test13a=xr.ones_like(sigs_array_99_cpc).where(slopes_array_99_cpc<0).fillna(0)
test14a=xr.ones_like(sigs_array_99_era5).where(slopes_array_99_era5<0).fillna(0)
test15a=xr.ones_like(sigs_array_99_jra55).where(slopes_array_99_jra55<0).fillna(0)
test16a=xr.ones_like(sigs_array_99_merra2).where(slopes_array_99_merra2<0).fillna(0)

test17=xr.ones_like(sigs_array_98_mswep).where(slopes_array_98_mswep>0).fillna(0)
test18=xr.ones_like(sigs_array_98_persiann).where(slopes_array_98_persiann>0).fillna(0)
test19=xr.ones_like(sigs_array_98_regen).where(slopes_array_98_regen>0).fillna(0)
test20=xr.ones_like(sigs_array_98_gpcc).where(slopes_array_98_gpcc>0).fillna(0)
test21=xr.ones_like(sigs_array_98_cpc).where(slopes_array_98_cpc>0).fillna(0)
test22=xr.ones_like(sigs_array_98_era5).where(slopes_array_98_era5>0).fillna(0)
test23=xr.ones_like(sigs_array_98_jra55).where(slopes_array_98_jra55>0).fillna(0)
test24=xr.ones_like(sigs_array_98_merra2).where(slopes_array_98_merra2>0).fillna(0)


test_all=test1+test2+test3+test4+test5+test6+test7+test8+test9+test10+test11+test12+test13+test14+test15+test16+test17+test18+test19+test20+test21+test22+test23+test24
field=test_all/(num_sets*3.)


test17a=xr.ones_like(sigs_array_98_mswep).where(slopes_array_98_mswep<0).fillna(0)
test18a=xr.ones_like(sigs_array_98_persiann).where(slopes_array_98_persiann<0).fillna(0)
test19a=xr.ones_like(sigs_array_98_regen).where(slopes_array_98_regen<0).fillna(0)
test20a=xr.ones_like(sigs_array_98_gpcc).where(slopes_array_98_gpcc<0).fillna(0)
test21a=xr.ones_like(sigs_array_98_cpc).where(slopes_array_98_cpc<0).fillna(0)
test22a=xr.ones_like(sigs_array_98_era5).where(slopes_array_98_era5<0).fillna(0)
test23a=xr.ones_like(sigs_array_98_jra55).where(slopes_array_98_jra55<0).fillna(0)
test24a=xr.ones_like(sigs_array_98_merra2).where(slopes_array_98_merra2<0).fillna(0)


test_alla=test1a+test2a+test3a+test4a+test5a+test6a+test7a+test8a+test9a+test10a+test11a+test12a+test13a+test14a+test15a+test16a+test17a+test18a+test19a+test20a+test21a+test22a+test23a+test24a
fielda=test_alla/(num_sets*3.)


from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,0.6,128))
colors2=plt.cm.Blues_r(np.linspace(0.4,1,128))

colors3=LinearSegmentedColormap.from_list('colors3',[colors1[-1],'orange'])
colors3=colors3(np.linspace(0,1,64))
colors4=LinearSegmentedColormap.from_list('colors4',['darkviolet',colors2[0]])
colors4=colors4(np.linspace(0,1,64))

colors=np.vstack((colors4,colors2,colors1,colors3))

cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field.where(field>1./2.),levels=np.arange(-1,1.1,0.1),extend='both',cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25)

cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>1./2.)*-1,levels=np.arange(-1,1.1,0.1),extend='both',cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=36) 
cbar.set_label('Fraction of datasets',fontsize=42,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':36,'color':'k'}
g1.ylabel_style={'size':36,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{EP}$'+' '+r'$\bf{regression}$'+' '+r'$\bf{slope}$'+' '+r'$\bf{majorities, }$'+' '+r'$\bf{1990-2022}$',fontsize=48,pad=5)
#EP total regression slope: +2/3 positive/negative
plt.show()

In [ ]:
fig.savefig(dir_data+'num_datasets_1a.png')

In [ ]:
mean=total_95_mswep.mean(dim='time_mswep')
test1=slopes_array_95_mswep/mean*100.
test1=test1.fillna(0)

mean=total_95_persiann.mean(dim='time_persiann')
test2=slopes_array_95_persiann/mean*100.
test2=test2.fillna(0)

mean=total_95_regen.mean(dim='time_regen')
test3=slopes_array_95_regen/mean*100.
test3=test3.fillna(0)

mean=total_95_gpcc.mean(dim='time_gpcc')
test4=slopes_array_95_gpcc/mean*100.
test4=test4.fillna(0)

mean=total_95_cpc.mean(dim='time')
test5=slopes_array_95_cpc/mean*100.
test5=test5.fillna(0)

mean=total_95_era5.mean(dim='time')
test6=slopes_array_95_era5/mean*100.
test6=test6.fillna(0)

mean=total_95_jra55.mean(dim='time')
test7=slopes_array_95_jra55/mean*100.
test7=test7.fillna(0)

mean=total_95_merra2.mean(dim='time')
test8=slopes_array_95_merra2/mean*100.
test8=test8.fillna(0)




mean=total_99_mswep.mean(dim='time_mswep')
test9=slopes_array_99_mswep/mean*100.
test9=test9.fillna(0)

mean=total_99_persiann.mean(dim='time_persiann')
test10=slopes_array_99_persiann/mean*100.
test10=test10.fillna(0)

mean=total_99_regen.mean(dim='time_regen')
test11=slopes_array_99_regen/mean*100.
test11=test11.fillna(0)

mean=total_99_gpcc.mean(dim='time_gpcc')
test12=slopes_array_99_gpcc/mean*100.
test12=test12.fillna(0)

mean=total_99_cpc.mean(dim='time')
test13=slopes_array_99_cpc/mean*100.
test13=test13.fillna(0)

mean=total_99_era5.mean(dim='time')
test14=slopes_array_99_era5/mean*100.
test14=test14.fillna(0)

mean=total_99_jra55.mean(dim='time')
test15=slopes_array_99_jra55/mean*100.
test15=test15.fillna(0)

mean=total_99_merra2.mean(dim='time')
test16=slopes_array_99_merra2/mean*100.
test16=test16.fillna(0)




mean=total_98_mswep.mean(dim='time_mswep')
test17=slopes_array_98_mswep/mean*100.
test17=test17.fillna(0)

mean=total_98_persiann.mean(dim='time_persiann')
test18=slopes_array_98_persiann/mean*100.
test18=test18.fillna(0)

mean=total_98_regen.mean(dim='time_regen')
test19=slopes_array_98_regen/mean*100.
test19=test19.fillna(0)

mean=total_98_gpcc.mean(dim='time_gpcc')
test20=slopes_array_98_gpcc/mean*100.
test20=test20.fillna(0)

mean=total_98_cpc.mean(dim='time')
test21=slopes_array_98_cpc/mean*100.
test21=test21.fillna(0)

mean=total_98_era5.mean(dim='time')
test22=slopes_array_98_era5/mean*100.
test22=test22.fillna(0)

mean=total_98_jra55.mean(dim='time')
test23=slopes_array_98_jra55/mean*100.
test23=test23.fillna(0)

mean=total_98_merra2.mean(dim='time')
test24=slopes_array_98_merra2/mean*100.
test24=test24.fillna(0)



test_all=test1+test2+test3+test4+test5+test6+test7+test8+test9+test10+test11+test12+test13+test14+test15+test16+test17+test18+test19+test20+test21+test22+test23+test24
field=test_all/(num_sets*3.)


from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,0.6,128))
colors2=plt.cm.Blues_r(np.linspace(0.4,1,128))

colors3=LinearSegmentedColormap.from_list('colors3',[colors1[-1],'orange'])
colors3=colors3(np.linspace(0,1,64))
colors4=LinearSegmentedColormap.from_list('colors4',['darkviolet',colors2[0]])
colors4=colors4(np.linspace(0,1,64))

colors=np.vstack((colors4,colors2,colors1,colors3))

cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field,levels=np.arange(-2,2.2,0.2),extend='both',cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25)

#cax=ax.contourf(field.lon,field.lat,fielda.where(fielda>1./2.)*-1,levels=np.arange(-1,1.1,0.1),extend='both',cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=36) 
cbar.set_label('% yr$^{-1}$',fontsize=42,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':36,'color':'k'}
g1.ylabel_style={'size':36,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{EP}$'+' '+r'$\bf{mean}$'+' '+r'$\bf{regression}$'+' '+r'$\bf{slope, }$'+' '+r'$\bf{1990-2022}$',fontsize=48,pad=5)
#EP total regression slope: +2/3 positive/negative
plt.show()

In [ ]:
fig.savefig(dir_data+'num_datasets_2a.png')

In [ ]:
test1=xr.ones_like(sigs_array_95_mswep).where(sigs_array_95_mswep<=p_final_95_mswep).where(slopes_array_95_mswep>0).fillna(0)
test2=xr.ones_like(sigs_array_95_persiann).where(sigs_array_95_persiann<=p_final_95_persiann).where(slopes_array_95_persiann>0).fillna(0)
test3=xr.ones_like(sigs_array_95_regen).where(sigs_array_95_regen<=p_final_95_regen).where(slopes_array_95_regen>0).fillna(0)
test4=xr.ones_like(sigs_array_95_gpcc).where(sigs_array_95_gpcc<=p_final_95_gpcc).where(slopes_array_95_gpcc>0).fillna(0)
test5=xr.ones_like(sigs_array_95_cpc).where(sigs_array_95_cpc<=p_final_95_cpc).where(slopes_array_95_cpc>0).fillna(0)
test6=xr.ones_like(sigs_array_95_era5).where(sigs_array_95_era5<=p_final_95_era5).where(slopes_array_95_era5>0).fillna(0)
test7=xr.ones_like(sigs_array_95_jra55).where(sigs_array_95_jra55<=p_final_95_jra55).where(slopes_array_95_jra55>0).fillna(0)
test8=xr.ones_like(sigs_array_95_merra2).where(sigs_array_95_merra2<=p_final_95_merra2).where(slopes_array_95_merra2>0).fillna(0)

test1a=xr.ones_like(sigs_array_95_mswep).where(sigs_array_95_mswep<=p_final_95_mswep).where(slopes_array_95_mswep<0).fillna(0)
test2a=xr.ones_like(sigs_array_95_persiann).where(sigs_array_95_persiann<=p_final_95_persiann).where(slopes_array_95_persiann<0).fillna(0)
test3a=xr.ones_like(sigs_array_95_regen).where(sigs_array_95_regen<=p_final_95_regen).where(slopes_array_95_regen<0).fillna(0)
test4a=xr.ones_like(sigs_array_95_gpcc).where(sigs_array_95_gpcc<=p_final_95_gpcc).where(slopes_array_95_gpcc<0).fillna(0)
test5a=xr.ones_like(sigs_array_95_cpc).where(sigs_array_95_cpc<=p_final_95_cpc).where(slopes_array_95_cpc<0).fillna(0)
test6a=xr.ones_like(sigs_array_95_era5).where(sigs_array_95_era5<=p_final_95_era5).where(slopes_array_95_era5<0).fillna(0)
test7a=xr.ones_like(sigs_array_95_jra55).where(sigs_array_95_jra55<=p_final_95_jra55).where(slopes_array_95_jra55<0).fillna(0)
test8a=xr.ones_like(sigs_array_95_merra2).where(sigs_array_95_merra2<=p_final_95_merra2).where(slopes_array_95_merra2<0).fillna(0)

test9=xr.ones_like(sigs_array_99_mswep).where(sigs_array_99_mswep<=p_final_99_mswep).where(slopes_array_99_mswep>0).fillna(0)
test10=xr.ones_like(sigs_array_99_persiann).where(sigs_array_99_persiann<=p_final_99_persiann).where(slopes_array_99_persiann>0).fillna(0)
test11=xr.ones_like(sigs_array_99_regen).where(sigs_array_99_regen<=p_final_99_regen).where(slopes_array_99_regen>0).fillna(0)
test12=xr.ones_like(sigs_array_99_gpcc).where(sigs_array_99_gpcc<=p_final_99_gpcc).where(slopes_array_99_gpcc>0).fillna(0)
test13=xr.ones_like(sigs_array_99_cpc).where(sigs_array_99_cpc<=p_final_99_cpc).where(slopes_array_99_cpc>0).fillna(0)
test14=xr.ones_like(sigs_array_99_era5).where(sigs_array_99_era5<=p_final_99_era5).where(slopes_array_99_era5>0).fillna(0)
test15=xr.ones_like(sigs_array_99_jra55).where(sigs_array_99_jra55<=p_final_99_jra55).where(slopes_array_99_jra55>0).fillna(0)
test16=xr.ones_like(sigs_array_99_merra2).where(sigs_array_99_merra2<=p_final_99_merra2).where(slopes_array_99_merra2>0).fillna(0)

test9a=xr.ones_like(sigs_array_99_mswep).where(sigs_array_99_mswep<=p_final_99_mswep).where(slopes_array_99_mswep<0).fillna(0)
test10a=xr.ones_like(sigs_array_99_persiann).where(sigs_array_99_persiann<=p_final_99_persiann).where(slopes_array_99_persiann<0).fillna(0)
test11a=xr.ones_like(sigs_array_99_regen).where(sigs_array_99_regen<=p_final_99_regen).where(slopes_array_99_regen<0).fillna(0)
test12a=xr.ones_like(sigs_array_99_gpcc).where(sigs_array_99_gpcc<=p_final_99_gpcc).where(slopes_array_99_gpcc<0).fillna(0)
test13a=xr.ones_like(sigs_array_99_cpc).where(sigs_array_99_cpc<=p_final_99_cpc).where(slopes_array_99_cpc<0).fillna(0)
test14a=xr.ones_like(sigs_array_99_era5).where(sigs_array_99_era5<=p_final_99_era5).where(slopes_array_99_era5<0).fillna(0)
test15a=xr.ones_like(sigs_array_99_jra55).where(sigs_array_99_jra55<=p_final_99_jra55).where(slopes_array_99_jra55<0).fillna(0)
test16a=xr.ones_like(sigs_array_99_merra2).where(sigs_array_99_merra2<=p_final_99_merra2).where(slopes_array_99_merra2<0).fillna(0)

test17=xr.ones_like(sigs_array_98_mswep).where(sigs_array_98_mswep<=p_final_98_mswep).where(slopes_array_98_mswep>0).fillna(0)
test18=xr.ones_like(sigs_array_98_persiann).where(sigs_array_98_persiann<=p_final_98_persiann).where(slopes_array_98_persiann>0).fillna(0)
test19=xr.ones_like(sigs_array_98_regen).where(sigs_array_98_regen<=p_final_98_regen).where(slopes_array_98_regen>0).fillna(0)
test20=xr.ones_like(sigs_array_98_gpcc).where(sigs_array_98_gpcc<=p_final_98_gpcc).where(slopes_array_98_gpcc>0).fillna(0)
test21=xr.ones_like(sigs_array_98_cpc).where(sigs_array_98_cpc<=p_final_98_cpc).where(slopes_array_98_cpc>0).fillna(0)
test22=xr.ones_like(sigs_array_98_era5).where(sigs_array_98_era5<=p_final_98_era5).where(slopes_array_98_era5>0).fillna(0)
test23=xr.ones_like(sigs_array_98_jra55).where(sigs_array_98_jra55<=p_final_98_jra55).where(slopes_array_98_jra55>0).fillna(0)
test24=xr.ones_like(sigs_array_98_merra2).where(sigs_array_98_merra2<=p_final_98_merra2).where(slopes_array_98_merra2>0).fillna(0)


test_all=test1+test2+test3+test4+test5+test6+test7+test8+test9+test10+test11+test12+test13+test14+test15+test16+test17+test18+test19+test20+test21+test22+test23+test24
field=test_all/(num_sets*3.)


test17a=xr.ones_like(sigs_array_98_mswep).where(sigs_array_98_mswep<=p_final_98_mswep).where(slopes_array_98_mswep<0).fillna(0)
test18a=xr.ones_like(sigs_array_98_persiann).where(sigs_array_98_persiann<=p_final_98_persiann).where(slopes_array_98_persiann<0).fillna(0)
test19a=xr.ones_like(sigs_array_98_regen).where(sigs_array_98_regen<=p_final_98_regen).where(slopes_array_98_regen<0).fillna(0)
test20a=xr.ones_like(sigs_array_98_gpcc).where(sigs_array_98_gpcc<=p_final_98_gpcc).where(slopes_array_98_gpcc<0).fillna(0)
test21a=xr.ones_like(sigs_array_98_cpc).where(sigs_array_98_cpc<=p_final_98_cpc).where(slopes_array_98_cpc<0).fillna(0)
test22a=xr.ones_like(sigs_array_98_era5).where(sigs_array_98_era5<=p_final_98_era5).where(slopes_array_98_era5<0).fillna(0)
test23a=xr.ones_like(sigs_array_98_jra55).where(sigs_array_98_jra55<=p_final_98_jra55).where(slopes_array_98_jra55<0).fillna(0)
test24a=xr.ones_like(sigs_array_98_merra2).where(sigs_array_98_merra2<=p_final_98_merra2).where(slopes_array_98_merra2<0).fillna(0)


test_alla=test1a+test2a+test3a+test4a+test5a+test6a+test7a+test8a+test9a+test10a+test11a+test12a+test13a+test14a+test15a+test16a+test17a+test18a+test19a+test20a+test21a+test22a+test23a+test24a
fielda=test_alla/(num_sets*3.)

field_up=np.where(field.values>fielda.values,field,np.nan)
field_upa=np.where(fielda.values>field.values,fielda,np.nan)


from matplotlib.colors import LinearSegmentedColormap
colors1=plt.cm.Reds(np.linspace(0,1,128))
colors2=plt.cm.Blues_r(np.linspace(0,1,128))
colors=np.vstack((colors2,colors1))
cmap_new=LinearSegmentedColormap.from_list('cmap_new',colors)

fig=plt.figure(figsize=(24,8))#24,12
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.contourf(field.lon,field.lat,field_up,levels=np.arange(-1,1.1,0.1),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25)

cax=ax.contourf(field.lon,field.lat,field_upa*-1,levels=np.arange(-1,1.1,0.1),cmap=cmap_new,transform=ccrs.PlateCarree(),alpha=1.0)   
#cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25,ticks=np.arange(1,10,1))


g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=36) 
cbar.set_label('Fraction of datasets',fontsize=42,labelpad=10)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':36,'color':'k'}
g1.ylabel_style={'size':36,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False
#g1.left_labels=False
#g1.bottom_labels=False

g1.xlocator = mticker.FixedLocator([-120,-60,0,60,120])
            
ax.set_title(r'$\bf{EP}$'+' '+r'$\bf{trend}$'+' '+r'$\bf{consensus, }$'+' '+r'$\bf{1990-2022}$',fontsize=48,pad=5)

plt.show()

In [ ]:
fig.savefig(dir_data+'num_datasets_3a.png')